In [6]:
from __future__ import division, print_function

import numpy as np
from scipy.sparse import random
from scipy import stats
from numpy.random import default_rng
from pylab import plt

In [7]:
def J(Y, lamb):
    return max(np.linalg.norm(Y, ord=2), np.linalg.norm(Y, np.inf)/lamb)

In [8]:
class R_pca:

    def __init__(self, D, Y, mu=None, lmbda=None, rho=None):
        self.D = D
        self.E = np.zeros(self.D.shape)
        # self.Y = np.zeros(self.D.shape)
        self.Y = Y
        self.rho = rho

        if mu:
            self.mu = mu
        else:
            self.mu = np.prod(self.D.shape) / (4 * self.norm_p(self.D, 2))  

        self.mu_inv = 1 / self.mu

        if lmbda:
            self.lmbda = lmbda
        else:
            self.lmbda = 1 / np.sqrt(32*np.max(self.D.shape))

    @staticmethod
    def norm_p(M, p):
        return np.sum(np.power(M, p))

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self, tol=None, max_iter=1000, iter_print=100):
        iter = 0
        err = np.Inf
        Ek = self.E
        Yk = self.Y
        Ak = np.zeros(self.D.shape)

        if tol:
            _tol = tol
        else:
            _tol = 1E-7 * self.norm_p(np.abs(self.D), 2)

        while (err > _tol) and iter < max_iter:
            Ak = self.svd_threshold(self.D - Ek + self.mu_inv * Yk, self.mu_inv)
            Ek = self.shrink(self.D - Ak + (self.mu_inv * Yk), self.mu_inv * self.lmbda)
            Yk = Yk + self.mu * (self.D - Ak - Ek)
            err = self.norm_p(np.abs(self.D - Ak - Ek), 2)
            self.mu *= self.rho
            iter += 1

        np.fill_diagonal(Ak, 1)
        np.fill_diagonal(Ek, 0)

        self.A = Ak
        self.E = Ek
        return Ak, Ek

In [9]:
def relu(x):
    return np.maximum(0, x)

In [10]:
def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

def svd_threshold(M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(shrink(S, tau)), V))

In [12]:
def rpca_pg(D, lamb, mu):
    n = len(D)
    Ak = np.zeros(D.shape)
    Ak1 = np.zeros(D.shape)
    Ek = np.zeros(D.shape)
    Ek1 = np.zeros(D.shape)
    tk = 1
    tk1 = 1
    mu_0 = 0.99*np.linalg.norm(D, ord=2)
    mu_bar = mu_0/100000
    i = 1
    while(i < 10):
        Ak_tilda = Ak + (tk1-1)/tk*(Ak - Ak1)
        Ek_tilda = Ek + (tk1-1)/tk*(Ek - Ek1)
        YkA = Ak_tilda - 0.5*(Ak_tilda + Ek_tilda - D)
        U, S, V = np.linalg.svd(YkA, full_matrices=False)
        Ak1 = np.copy(Ak)
        Ek1 = np.copy(Ek)
        Ak = U*relu(S - 0.5*mu*np.identity(n))*V.T
        YkE = Ek_tilda - 0.5*(Ak_tilda + Ek_tilda - D)
        Ek = np.sign(YkE)*relu(np.linalg.det(YkE) - lamb*mu/2)
        tk1 = tk
        tk = (1+np.sqrt(1 + 4*tk*tk))/2
        mu = max(0.9*mu, mu_bar)
        i += 1
    return Ak, Ek

In [13]:
def InexactALM(D, lamb, mu, rho):
    Y0 = D/J(D, lamb)
    # Y0 = np.zeros(D.shape)
    rpca = R_pca(D, Y0, mu, lamb, rho)
    A, E = rpca.fit()
    return (A, E)

In [385]:
def accelerated_rpca_pg(D, lamb, delta, ita):
    n = len(D)
    mu_0 = ita * np.linalg.norm(D, ord=2)
    Ak = np.zeros(D.shape)
    A_k_minus1 = np.zeros(D.shape)
    Ek = np.zeros(D.shape)
    E_k_minus1 = np.zeros(D.shape)
    tk = 1
    t_k_minus1 = 1
    mu_bar = delta * mu_0
    mu_k = mu_0*0.99
    k = 0
    while(k < 50):
        Yk_A = Ak + (t_k_minus1 - 1)/tk*(Ak - A_k_minus1)
        Yk_E = Ek + (t_k_minus1 - 1)/tk*(Ek - E_k_minus1)
        Gk_A = Yk_A - 0.5*(Yk_A + Yk_E - D)
        A_k_plus1 = svd_threshold(Gk_A, mu_k/2)
        Gk_E = Yk_E - 0.5*(Yk_A + Yk_E - D)
        E_k_plus1 = shrink(Gk_E, lamb*mu_k/2)
        t_k_plus1 = 0.5 * (1 + np.sqrt(1 + 4*tk*tk))
        # mu_k_plus1 = max(ita * mu_k, mu_bar)
        mu_k = max(ita * mu_k, mu_bar)
        k += 1
        A_k_minus1 = Ak.copy()
        Ak = A_k_plus1.copy()
        E_k_minus1 = Ek.copy()
        Ek = E_k_plus1.copy()
        # mu_k = mu_k_plus1
        t_k_minus1 = tk
        tk = t_k_plus1

    return Ak, Ek


In [386]:
def Clustering(D):
    n = len(D)
    A = np.zeros(D.shape)
    lamb = 1/(32*np.sqrt(n))
    i = 0
    while(i < 10):
        # A, E = InexactALM(D+np.identity(n), lamb, mu=1.4, rho=1.1)
        # A, E = rpca_pg(D+np.identity(n), lamb, mu=0.005)
        A, E = accelerated_rpca_pg(D + np.identity(n), lamb, 0.001, 0.9)
        # if(A.trace() > n):
        #     lamb /= 2
        #     # print("0")
        # elif(A.trace() < n):
        #     lamb *= 2
        #     # print("1")
        # else:
        #     pass
        if (A.trace() < n):
            lamb *= 1.4
        i += 1
    A = np.around(A)
    E = np.around(E)
    A = A.astype(int)
    E = E.astype(int)
    np.fill_diagonal(A, 1)
    np.fill_diagonal(E, 0)
    print(A)
    print(E)
    print("Rank of A:", np.linalg.matrix_rank(A))
    print("Rank of E: ", np.linalg.matrix_rank(E))

In [398]:
## ChatGPT1

# import numpy as np

# def svt(M, tau, max_iter=100, tol=1e-6):
#     """
#     Performs singular value thresholding on a matrix M with threshold tau,
#     using max_iter iterations and a convergence tolerance of tol.
#     """
#     Y = M.copy()
#     for i in range(max_iter):
#         U, sigma, Vt = np.linalg.svd(Y)
#         sigma_t = np.maximum(sigma - tau, 0)
#         L = np.dot(U, np.dot(np.diag(sigma_t), Vt))
#         if np.linalg.norm(L - Y) / np.linalg.norm(Y) < tol:
#             break
#         Y = L
#     return L

# def split_matrix(M, tau):
#     """
#     Splits a symmetric matrix M into a low-rank matrix L and a sparse matrix S,
#     such that M = L + S and L has rank <= tau.
#     """
#     # Perform singular value thresholding on M to get a low-rank approximation
#     L = svt(M, tau)
    
#     # Construct the sparse matrix S
#     S = M - L

#     S = np.around(S)
#     L = np.around(L)
#     S = S.astype(int)
#     L = L.astype(int)
    
#     return L, S


In [412]:
### chat GPT2


# import numpy as np

# def svt(M, tau):
#     """
#     Computes the singular value thresholding (SVT) of a matrix M with threshold tau.
#     """
#     U, sigma, Vt = np.linalg.svd(M, full_matrices=False)
#     sigma_t = np.maximum(sigma - tau, 0)
#     return U @ np.diag(sigma_t) @ Vt

# def split_matrix(M, tol=1e-6, max_iter=100):
#     """
#     Splits a symmetric matrix M into a low-rank matrix L and a sparse matrix S,
#     such that M = L + S and L has rank <= tau.
#     """
#     n = M.shape[0]
#     L = np.zeros((n, n))
#     S = M.copy()
#     Y = np.zeros((n, n))
#     converged = False
#     for i in range(max_iter):
#         L = svt(S + Y, 1)
#         S = M - L
#         Y += S
#         if np.linalg.norm(S, 'fro') / np.linalg.norm(M, 'fro') < tol:
#             converged = True
#             break
#     if not converged:
#         print('Warning: SVT did not converge.')


#     S = np.around(S)
#     L = np.around(L)
#     S = S.astype(int)
#     L = L.astype(int)
#     return L, S


In [420]:
### chat gpt 3

def estimate_rank(M):
    """
    Heuristic to estimate the rank of a matrix based on the singular values.
    """
    U, s, Vt = svd(M)
    tol = np.finfo(float).eps * max(M.shape) * max(s)
    rank = np.sum(s > tol)
    return rank

import numpy as np
from scipy.linalg import svd

def alm_matrix_split(M, rank, rho=1.0, alpha=1.0, max_iter=1000, tol=1e-6):
    """
    Computes a low-rank and sparse matrix approximation of M using the
    augmented Lagrange multiplier (ALM) method with parameters rho and alpha.
    """
    n = M.shape[0]
    X = np.zeros((n, n))
    Z = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(max_iter):
        # Update X using the SVD
        U, sigma, Vt = svd(Z - U / rho, full_matrices=False)
        sigma_th = (sigma - alpha / rho).clip(min=0)
        X = U.dot(np.diag(sigma_th)).dot(Vt)

        # Update Z and U
        Z_old = Z.copy()
        Z = X + U / rho
        Z = (Z + Z.T) / 2
        U = U + rho * (X - Z)

        # Check convergence
        err = np.linalg.norm(X - Z, ord='fro')
        if err < tol:
            break

    L = X
    S = M - L


    S = np.around(S)
    L = np.around(L)
    S = S.astype(int)
    L = L.astype(int)

    return L, S


In [430]:
if __name__=="__main__":
    n = 8
    p = 0.8
    A1 = np.random.binomial(n=1, p=p, size=(n,n))
    A = (A1 + A1.T)/2
    A = np.around(A)
    A = A.astype(int)
    np.fill_diagonal(A, 0)
    print(A)
    print(np.linalg.matrix_rank(A))
    Clustering(A)
    # A, E = alm_matrix_split(A, estimate_rank(A))
    

[[0 0 1 1 0 1 1 1]
 [0 0 0 1 1 1 1 1]
 [1 0 0 1 0 1 0 0]
 [1 1 1 0 0 1 0 1]
 [0 1 0 0 0 1 1 1]
 [1 1 1 1 1 0 1 0]
 [1 1 0 0 1 1 0 1]
 [1 1 0 1 1 0 1 0]]
7
[[1 0 0 0 0 0 0 0]
 [0 1 0 0 1 1 1 1]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 1 0 0 1 1 1 1]
 [0 1 0 0 1 1 1 1]
 [0 1 0 0 1 1 1 1]
 [0 1 0 0 1 1 1 1]]
[[ 0  0  1  1  0  1  1  1]
 [ 0  0  0  1  0  0  0  0]
 [ 1  0  0  1  0  1  0  0]
 [ 1  1  1  0  0  1  0  1]
 [ 0  0  0  0  0  0  0  0]
 [ 1  0  1  1  0  0  0 -1]
 [ 1  0  0  0  0  0  0  0]
 [ 1  0  0  1  0 -1  0  0]]
Rank of A: 4
Rank of E:  6


In [77]:
def is_block_diagonal(matrix):
    # print(matrix)
    n = len(matrix)
    block_size = None
    col_starts = []
    for i in range(n):
        if block_size is None:
            block_size = len(matrix[i])
        elif len(matrix[i]) != block_size:
            # print("1234")
            return False
        col_starts.append(i*block_size)
    for i in range(n):
        start = col_starts[i]
        end = start + block_size
        for j in range(start, end):
            # print(matrix[i, j-start])
            if matrix[i, j-start] != 0 and any(matrix[k, j-start] != 0 for k in range(n) if k != i):
                print("5678")
                return False
    return True


In [80]:
p = 0.8
A1 = np.random.binomial(n=1, p=p, size=(n,n))
A = (A1 + A1.T)/2
A = np.around(A)
A = A.astype(int)
np.fill_diagonal(A, 0)
# A = [[1, 1, 0, 0], [1, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 1]]
B = np.matrix([[1, 1, 0, 0], [1, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 1]])
print(B)
print(is_block_diagonal(B))

[[1 1 0 0]
 [1 1 0 0]
 [0 0 1 1]
 [0 0 1 1]]
1
5678
False
